In [24]:
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import urlopen
import re


In [25]:
def create_full_urls(links):
    for i in range(len(links)):
        extension = links[i]
        full_link = 'https://www.aisixiang.com' + extension
        links[i] = full_link

    return links

def get_links_of_type(soup, link_substring):
    """Return list of links from the page that have the given substring"""

    link_eles = soup.find_all('a')
    all_links = [a['href'] for a in link_eles]

    filtered_links = [link for link in all_links if link_substring in link]

    links = create_full_urls(filtered_links)

    return links


root_page = urlopen('https://www.aisixiang.com/thinktank/')
root_html = root_page.read().decode("utf-8")
root_soup = BeautifulSoup(root_html, "html.parser")

author_links = get_links_of_type(root_soup, 'thinktank')
author_links.pop(0)
len(author_links)

810

In [30]:
from tqdm import tqdm
from urllib.error import URLError

In [31]:
article_links = []
failed_urls = []

def get_article_links(timeout=10):
    failed_urls = []

    for link in tqdm(author_links):
        try:
            with urlopen(link, timeout=timeout) as author_page:
                author_html = author_page.read().decode("utf-8")
                author_soup = BeautifulSoup(author_html, "html.parser")

                author_article_links = get_links_of_type(author_soup, 'data')
                article_links.extend(author_article_links)
        except URLError as e:
            print(f"Error fetching links from {link}: {e}")
            failed_urls.append(link)
        except Exception as e:
            print(f"Error processing {link}: {e}")
            failed_urls.append(link)

    return failed_urls


In [32]:
failed_urls = get_article_links()

100%|██████████| 810/810 [03:24<00:00,  3.97it/s]


In [34]:
len(article_links)

56341

In [35]:
article_links_df = pd.DataFrame({'links': article_links})

In [36]:
article_links_df.head(5)

,links
0,https://www.aisixiang.com/data/149906.html
1,https://www.aisixiang.com/data/149797.html
2,https://www.aisixiang.com/data/149796.html
3,https://www.aisixiang.com/data/149762.html
4,https://www.aisixiang.com/data/149760.html


In [37]:
article_links_df.to_csv('asx_article_links.csv', index=False)